In [504]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import validation_curve
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [505]:
%pwd

'/Users/terryjiang'

In [506]:
bank_bal=pd.read_csv('data.csv.zip')
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.493687,0.539468,0.543230,0.604455,0.604462,0.998992,0.797409,0.809331,0.303510,...,0.799927,0.000466,0.623620,0.604455,0.840359,0.279606,0.027064,0.566193,1,0.029890
6815,0,0.475162,0.538269,0.524172,0.598308,0.598308,0.998992,0.797414,0.809327,0.303520,...,0.799748,0.001959,0.623931,0.598306,0.840306,0.278132,0.027009,0.566018,1,0.038284
6816,0,0.472725,0.533744,0.520638,0.610444,0.610213,0.998984,0.797401,0.809317,0.303512,...,0.797778,0.002840,0.624156,0.610441,0.840138,0.275789,0.026791,0.565158,1,0.097649
6817,0,0.506264,0.559911,0.554045,0.607850,0.607850,0.999074,0.797500,0.809399,0.303498,...,0.811808,0.002837,0.623957,0.607846,0.841084,0.277547,0.026822,0.565302,1,0.044009


# Data Sampling

In [507]:
bank_bal=bank_bal.groupby('Bankrupt?',group_keys=False).apply(lambda x: x.sample(1000,replace=True,random_state=0))
bank_bal['Bankrupt?'].value_counts()

0    1000
1    1000
Name: Bankrupt?, dtype: int64

In [508]:
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2893,0,0.616195,0.655037,0.642915,0.616325,0.616325,0.999192,0.797712,0.809545,0.303622,...,0.861914,0.002909,0.623981,0.616320,0.844897,0.278677,0.026794,0.565174,1,0.034458
2765,0,0.582850,0.621784,0.622196,0.642586,0.642586,0.999160,0.797665,0.809535,0.303607,...,0.844453,0.000395,0.623838,0.642585,0.842776,0.276194,0.026802,0.565209,1,0.074229
1731,0,0.500463,0.545464,0.547620,0.605198,0.605198,0.999033,0.797413,0.809341,0.303432,...,0.799789,0.001395,0.408682,0.605198,0.840362,0.280128,0.028611,0.568346,1,0.028003
3434,0,0.475455,0.545464,0.532684,0.598661,0.598675,0.998970,0.797396,0.809328,0.303534,...,0.802893,0.000755,0.623876,0.598659,0.840796,0.283750,0.027123,0.566364,1,0.020906
5125,0,0.518403,0.578772,0.562129,0.599158,0.599165,0.999015,0.797446,0.809354,0.303527,...,0.819451,0.005034,0.624249,0.599160,0.842552,0.283468,0.026844,0.565399,1,0.021249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,1,0.412616,0.422209,0.453557,0.598358,0.598358,0.998916,0.796831,0.808784,0.302660,...,0.678242,0.008544,0.623109,0.598354,0.829094,0.281469,0.026637,0.564297,1,0.024493
4264,1,0.407400,0.442652,0.438086,0.609349,0.609320,0.998786,0.797106,0.808996,0.303413,...,0.727838,0.000782,0.623797,0.609344,0.831255,0.286656,0.026645,0.564353,1,0.018316
1752,1,0.448447,0.463694,0.486000,0.594027,0.594027,0.998887,0.797178,0.809074,0.303328,...,0.748700,0.001631,0.624708,0.594027,0.834098,0.286193,0.026639,0.564309,1,0.018641
2704,1,0.495539,0.545901,0.547674,0.619770,0.619770,0.999041,0.797473,0.809367,0.303521,...,0.800066,0.002033,0.625247,0.619771,0.840303,0.277260,0.027737,0.567538,1,0.047776


# Logistic Regression¶

In [509]:
bank_bal[bank_bal['Bankrupt?']==1].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,1.0,0.42,0.45,0.46,0.60,0.60,1.00,0.80,0.81,0.30,...,0.73,1.276100e+08,0.62,0.60,0.82,0.29,0.03,0.56,1.0,0.02
std,0.0,0.08,0.11,0.09,0.01,0.01,0.00,0.00,0.01,0.01,...,0.09,1.012655e+09,0.01,0.01,0.08,0.07,0.01,0.01,0.0,0.04
min,1.0,0.02,0.00,0.03,0.53,0.53,0.99,0.76,0.76,0.24,...,0.41,0.000000e+00,0.59,0.53,0.00,0.00,0.02,0.55,1.0,0.00
25%,1.0,0.39,0.43,0.43,0.59,0.59,1.00,0.80,0.81,0.30,...,0.72,0.000000e+00,0.62,0.59,0.83,0.28,0.03,0.56,1.0,0.02
50%,1.0,0.44,0.49,0.49,0.60,0.60,1.00,0.80,0.81,0.30,...,0.76,0.000000e+00,0.62,0.60,0.84,0.29,0.03,0.56,1.0,0.02
75%,1.0,0.47,0.53,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.79,1.000000e-02,0.62,0.60,0.84,0.29,0.03,0.57,1.0,0.02
max,1.0,0.58,0.58,0.62,0.67,0.67,1.00,0.80,0.81,0.31,...,0.82,9.170000e+09,0.68,0.67,1.00,1.00,0.26,0.62,1.0,0.80


In [510]:
bank_bal[bank_bal['Bankrupt?']==0].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,1.651000e+07,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
std,0.0,0.06,0.06,0.06,0.01,0.01,0.00,0.00,0.00,0.00,...,0.03,3.742320e+08,0.01,0.01,0.00,0.01,0.02,0.02,0.0,0.05
min,0.0,0.23,0.16,0.21,0.45,0.45,0.97,0.75,0.77,0.28,...,0.53,0.000000e+00,0.41,0.45,0.78,0.27,0.00,0.17,1.0,0.01
25%,0.0,0.48,0.54,0.53,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.000000e+00,0.62,0.60,0.84,0.28,0.03,0.57,1.0,0.02
50%,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.000000e+00,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.03
75%,0.0,0.54,0.59,0.59,0.61,0.61,1.00,0.80,0.81,0.30,...,0.83,1.000000e-02,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
max,0.0,1.00,0.95,0.96,0.66,0.66,1.00,0.80,0.81,0.31,...,1.00,9.650000e+09,0.73,0.66,0.85,0.40,0.54,1.00,1.0,0.94


## controlling the complexity of log regression, using parameter 'c'

In [511]:
bank_bal=bank_bal.sample(frac=1, random_state=0)
bank_bal.head(1)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
4191,0,0.372934,0.431313,0.414904,0.594322,0.594135,0.998888,0.797214,0.809148,0.303388,...,0.723812,0.005107,0.623694,0.594319,0.828935,0.290253,0.026745,0.564931,1,0.016448


In [512]:
X_bank, y_bank=bank_bal.iloc[:,1:], bank_bal['Bankrupt?'] 
display(X_bank.head(2))
display(y_bank.head(2))

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
4191,0.372934,0.431313,0.414904,0.594322,0.594135,0.998888,0.797214,0.809148,0.303388,0.781377,...,0.723812,0.005107,0.623694,0.594319,0.828935,0.290253,0.026745,0.564931,1,0.016448
1447,0.479745,0.545028,0.530060,0.619020,0.618235,0.999088,0.797430,0.809334,0.303347,0.781579,...,0.798701,0.003045,0.623882,0.619021,0.840290,0.281135,0.028226,0.568060,1,0.025232


4191    0
1447    1
Name: Bankrupt?, dtype: int64

## Feature Selection

In [513]:
from sklearn.tree import DecisionTreeClassifier

In [514]:
dt=DecisionTreeClassifier(random_state=0)
dt.fit(X_bank,y_bank)

DecisionTreeClassifier(random_state=0)

In [515]:
sum(dt.feature_importances_>.01)

17

In [516]:
X_bank_sel=X_bank[X_bank.columns[dt.feature_importances_>.01]]
X_bank_sel.head()

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
4191,0.594135,0.998888,0.781377,9.000000e+09,0.458792,0.000339,0.630462,0.377531,0.081935,0.007732,0.283268,1.009751e-03,0.907492,4.260000e+09,0.903625,0.005107,0.016448
1447,0.618235,0.999088,0.781579,2.763712e-04,0.459837,0.000437,0.633790,0.379239,0.021774,0.020430,0.277213,3.670025e-03,0.932293,1.893589e-04,0.680898,0.003045,0.025232
2496,0.578165,0.998648,0.780017,7.370000e+08,0.458980,0.000331,0.625583,0.514239,0.017742,0.621377,0.276974,6.440000e+08,0.912139,3.165289e-04,0.049582,0.387361,0.011326
4323,0.607684,0.999049,0.781726,1.453551e-04,0.461195,0.000705,0.630818,0.371526,0.059355,0.001240,0.277134,4.630348e-03,0.948180,5.310000e+08,0.829895,0.001079,0.030965
5500,0.599648,0.998970,0.781359,9.620000e+09,0.460554,0.000379,0.629533,0.395686,0.048871,0.003990,0.276440,3.269891e-03,0.878007,2.000000e+07,0.683642,0.001614,0.014274


## transform the features

In [517]:
scaler1=MinMaxScaler()
X_bank_trns=scaler1.fit_transform(X_bank_sel)
X_bank_trns

array([[0.67029143, 0.97094304, 0.90776909, ..., 0.89859766, 0.        ,
        0.01331719],
       [0.78217954, 0.97857407, 0.9117358 , ..., 0.66425108, 0.        ,
        0.02267398],
       [0.59614548, 0.96180078, 0.88109493, ..., 0.        , 0.        ,
        0.00786133],
       ...,
       [0.7761234 , 0.98249092, 0.91249831, ..., 0.8603324 , 0.        ,
        0.05813957],
       [0.81436745, 0.98252177, 0.91479961, ..., 0.91978202, 0.        ,
        0.13811285],
       [0.78933985, 0.98319587, 0.91669075, ..., 0.6758211 , 0.        ,
        0.03470609]])

## validation curve functions

In [518]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_trns, y_bank, test_size=0.2, random_state=0)

In [519]:
train_scores_log_reg, test_scores_log_reg=validation_curve(LogisticRegression(penalty='l1', solver='saga', max_iter=100000), X_train, y_train, param_name='C',
                                           param_range=[0.01,0.05,1,5,10,20], cv=4)

In [520]:
train_scores_log_reg.mean(axis=1).round(4)

array([0.4975, 0.7677, 0.8321, 0.8621, 0.8602, 0.861 ])

In [521]:
test_scores_log_reg.mean(axis=1).round(4)

array([0.4975, 0.77  , 0.8212, 0.8525, 0.8525, 0.8544])

In [522]:
log_reg=LogisticRegression(max_iter=1000, C=100)
log_reg.fit(X_train, y_train)

LogisticRegression(C=100, max_iter=1000)

In [523]:
print('log reg acc on train: {:.2%}'.format(log_reg.score(X_train, y_train)))
print('log reg acc on test: {:.2%}'.format(log_reg.score(X_test, y_test)))

log reg acc on train: 85.75%
log reg acc on test: 85.25%


In [524]:
log_reg.coef_

array([[ -7.18913619,  13.97367307, -11.11917622,  -0.14566533,
        -17.27773196,   1.16929683,   2.64163736,   3.56718348,
         -2.44283466,   3.66979635,  -9.88782096,   3.48182077,
        -14.59778957,  -0.33038749,  -0.24396249,  -2.83724605,
        -43.8013668 ]])

In [525]:
p1=[0.2,0.4,0.5,0.1,0.4,0.3,0.2,0.5,0.2,0.5,0.5,0.2,0.2,0.4,0.5,0.2,0.1]
log_reg.predict([p1])

array([1])

In [526]:
import numpy as np
np.set_printoptions(suppress=True)

In [527]:
log_reg.predict_proba([p1]).round(4)

array([[0.0184, 0.9816]])

In [528]:
log_reg.classes_

array([0, 1])

In [529]:
log_reg.coef_.round(4)

array([[ -7.1891,  13.9737, -11.1192,  -0.1457, -17.2777,   1.1693,
          2.6416,   3.5672,  -2.4428,   3.6698,  -9.8878,   3.4818,
        -14.5978,  -0.3304,  -0.244 ,  -2.8372, -43.8014]])

In [530]:
log_reg.intercept_.round(4)

array([20.6723])

# SVM 

In [531]:
X_bank_trns

array([[0.67029143, 0.97094304, 0.90776909, ..., 0.89859766, 0.        ,
        0.01331719],
       [0.78217954, 0.97857407, 0.9117358 , ..., 0.66425108, 0.        ,
        0.02267398],
       [0.59614548, 0.96180078, 0.88109493, ..., 0.        , 0.        ,
        0.00786133],
       ...,
       [0.7761234 , 0.98249092, 0.91249831, ..., 0.8603324 , 0.        ,
        0.05813957],
       [0.81436745, 0.98252177, 0.91479961, ..., 0.91978202, 0.        ,
        0.13811285],
       [0.78933985, 0.98319587, 0.91669075, ..., 0.6758211 , 0.        ,
        0.03470609]])

In [532]:
svm1=LinearSVC(random_state=0)
svm1.fit(X_train,y_train)

LinearSVC(random_state=0)

In [533]:
svm1.coef_

array([[-2.30557782,  2.87136628, -1.20251867, -0.04226328, -2.6195562 ,
        -0.        ,  1.09229715,  2.53142646, -0.25718109,  0.88190789,
        -1.54483213,  0.97711229, -5.15250239, -0.05836523, -0.07016647,
        -0.68150931, -4.47405014]])

In [534]:
svm1.intercept_

array([3.98125752])

In [535]:
print('svm acc on train: {:.2%}'.format(svm1.score(X_train, y_train)))
print('svm acc on test: {:.2%}'.format(svm1.score(X_test, y_test)))

svm acc on train: 82.56%
svm acc on test: 81.50%


In [536]:
b=[[0.616,0.999,0.781,0.00001,0.4719,0.0005,0.6306,0.3696,0.0415,0.0022,0.2770,0.9532,0,1.1,0.7021,0.0029,0.0345]]

In [537]:
b_trns=scaler1.transform(b)
b_trns

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[ 0.77180473,  0.97523232,  0.9003833 , -0.        ,  0.46878615,
         0.        ,  0.28705302,  0.3696    ,  0.08202934,  0.        ,
         0.14518228,  0.        , -2.69984748,  0.        ,  0.68655891,
         0.        ,  0.0325461 ]])

In [538]:
svm1.predict(b_trns)

array([1])

In [539]:
svm1.intercept_.round(2)

array([3.98])

In [540]:
svm1.coef_.round(2)

array([[-2.31,  2.87, -1.2 , -0.04, -2.62, -0.  ,  1.09,  2.53, -0.26,
         0.88, -1.54,  0.98, -5.15, -0.06, -0.07, -0.68, -4.47]])

## controlling the complexity of svm by change parameter 'C'

In [541]:
LinearSVC()

LinearSVC()

In [542]:
C_range=[.1,1,5,10,50,100]

In [543]:
train_scores_SVM, test_scores_SVM=validation_curve(LinearSVC(random_state=0, max_iter=100000), X_bank_trns, y_bank, param_name='C',
                                           param_range=[.1,1,5,10,50,100], cv=4)

In [544]:
train_scores_SVM.round(4)

array([[0.7927, 0.7913, 0.7793, 0.8073],
       [0.8227, 0.8173, 0.816 , 0.8253],
       [0.8313, 0.826 , 0.8193, 0.8353],
       [0.8293, 0.8267, 0.8233, 0.8387],
       [0.832 , 0.828 , 0.8267, 0.8373],
       [0.8287, 0.826 , 0.8267, 0.838 ]])

In [545]:
print('ave cross in 4 folds for different C values:')
print('training:', train_scores_SVM.mean(axis=1).round(4))
print('test', test_scores_SVM.mean(axis=1).round(4))

ave cross in 4 folds for different C values:
training: [0.7927 0.8203 0.828  0.8295 0.831  0.8298]
test [0.792  0.819  0.825  0.823  0.8255 0.826 ]


In [546]:
train_scores_SVM.mean()

0.8218888888888888

# Decision tree

In [547]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel, y_bank, test_size=0.2, random_state=0)

In [548]:
X_train

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
2976,0.597580,0.998975,0.781352,5.720000e+09,0.462057,0.000375,0.626627,0.384970,0.020323,0.078628,0.276517,0.003928,0.926169,2.090000e+09,0.437606,0.024747,0.017852
1752,0.594027,0.998887,0.781314,8.930000e+09,0.464620,0.000445,0.630170,0.378416,0.045323,0.011180,0.276787,0.002764,0.909662,9.520000e+08,0.699852,0.001631,0.018641
2545,0.614321,0.998996,0.781608,3.055116e-04,0.470038,0.000492,0.630665,0.369637,0.123226,0.005275,0.272881,0.004779,0.943680,1.900000e+09,0.857087,0.017840,0.026585
2043,0.599281,0.998506,0.778385,5.547083e-04,0.460156,0.000388,0.629517,0.380517,0.010484,0.214833,0.278116,0.026081,0.917775,9.390000e+09,0.800271,0.013199,0.023393
2195,0.598589,0.998907,0.781569,1.418860e-04,0.463354,0.000445,0.634437,0.371937,0.020484,0.007471,0.277510,0.014832,0.892657,5.650000e+09,0.905755,0.000647,0.046597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,0.597789,0.998966,0.781567,7.040000e+09,0.454421,0.000444,0.632047,0.376490,0.044677,0.023803,0.276581,0.002457,0.938484,1.050000e+09,0.973240,0.006844,0.027471
283,0.607028,0.999047,0.781640,1.377231e-04,0.460396,0.000464,0.631275,0.374160,0.035645,0.006648,0.277619,0.019948,0.936673,6.380000e+09,0.658020,0.000516,0.029036
2001,0.540776,0.997789,0.780105,3.329175e-04,0.461074,0.000167,0.630547,0.279725,0.017903,0.042332,0.276956,0.000504,0.804014,2.629237e-04,0.998848,0.011098,0.008753
5398,0.606113,0.999038,0.781624,1.319412e-04,0.469084,0.000458,0.630800,0.370886,0.031774,0.010267,0.277113,0.001074,0.941676,1.075464e-04,0.740103,0.001225,0.026186


In [549]:
train_scores_decision_tree, test_scores_decision_tree=validation_curve(DecisionTreeClassifier(random_state=0), X_train, y_train, param_name='max_leaf_nodes', 
                                          param_range=[2,6,8,10,15,25,50], cv=5)

In [550]:
dt0=DecisionTreeClassifier(random_state=0, max_leaf_nodes=10)
dt0.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

In [551]:
print('dt0 acc on train: {:.2%}'.format(dt0.score(X_train, y_train)))
print('dt0 acc on test: {:.2%}'.format(dt0.score(X_test, y_test)))

dt0 acc on train: 90.25%
dt0 acc on test: 89.50%


In [552]:
p1=[0.616,0.999,0.781,0.0001,0.471,0.0005,0.631,0.3696,0.0415,0.0022,0.27698,0,0.953,0,0.7022,0.0029,0.0345]
dt0.predict([p1])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0])

## visulalzing our decision tree

In [553]:
export_graphviz(dt0, out_file='dt0_vis.dot', feature_names=X_bank_sel.columns, class_names=dt0.classes_.astype(str), impurity=False, filled=True)

In [554]:
dt0.classes_

array([0, 1])

## Feature Importance 

In [555]:
dt0.feature_importances_.round(3)

array([0.033, 0.   , 0.601, 0.   , 0.   , 0.053, 0.029, 0.141, 0.022,
       0.019, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.103])

In [556]:
X_bank_sel.columns

Index([' Realized Sales Gross Margin', ' Operating Profit Rate',
       ' Continuous interest rate (after tax)', ' Operating Expense Rate',
       ' Cash flow rate', ' Net Value Growth Rate', ' Interest Expense Ratio',
       ' Borrowing dependency', ' Net Worth Turnover Rate (times)',
       ' Allocation rate per person', ' Inventory/Working Capital',
       ' Inventory/Current Liability', ' Retained Earnings to Total Assets',
       ' Cash Turnover Rate', ' Current Liability to Liability',
       ' Total assets to GNP price', ' Equity to Liability'],
      dtype='object')

In [557]:
feat_imp=pd.DataFrame(data=dt0.feature_importances_.round(3),index=X_bank_sel.columns, columns=['importance'])
feat_imp.sort_values('importance', ascending=False)

,importance
Continuous interest rate (after tax),0.601
Borrowing dependency,0.141
Equity to Liability,0.103
Net Value Growth Rate,0.053
Realized Sales Gross Margin,0.033
Interest Expense Ratio,0.029
Net Worth Turnover Rate (times),0.022
Allocation rate per person,0.019
Cash flow rate,0.000
Operating Expense Rate,0.000


# Random Forest

In [558]:
bank_rf=RandomForestClassifier(n_estimators=300, random_state=0)
bank_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=0)

In [559]:
print('bank_rf on train: {:.2%}'.format(bank_rf.score(X_train, y_train)))
print('bank_rf on test: {:.2%}'.format(bank_rf.score(X_test, y_test)))

bank_rf on train: 100.00%
bank_rf on test: 96.50%


## Feature Importance

In [560]:
bank_rf.feature_importances_

array([0.03148619, 0.0427782 , 0.15396602, 0.01972965, 0.03710007,
       0.08806228, 0.08931111, 0.13006438, 0.02368479, 0.02624712,
       0.03605618, 0.02256591, 0.13319418, 0.02562697, 0.01899411,
       0.02206478, 0.09906807])

In [561]:
bank_rf_feat_imp=pd.DataFrame(data=bank_rf.feature_importances_, index=X_bank_sel.columns, columns=['importance'])
bank_rf_feat_imp.sort_values(by=['importance'], ascending=False).round(3)

,importance
Continuous interest rate (after tax),0.154
Retained Earnings to Total Assets,0.133
Borrowing dependency,0.130
Equity to Liability,0.099
Interest Expense Ratio,0.089
Net Value Growth Rate,0.088
Operating Profit Rate,0.043
Cash flow rate,0.037
Inventory/Working Capital,0.036
Realized Sales Gross Margin,0.031


In [562]:
X_test.iloc[20:25,:]

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
4927,0.601385,0.999041,0.781624,5.270000e+09,0.475860,0.000443,0.631241,0.375087,0.032258,0.084615,0.276399,0.002225,0.946754,1.310857e-04,0.579829,0.483133,0.027919
2521,0.590899,0.998823,0.781377,1.023383e-04,0.464033,0.000439,0.629082,0.382352,0.026935,0.043610,0.276975,0.000000,0.919474,1.826919e-04,0.511457,0.010901,0.019908
4558,0.609197,0.998994,0.781597,2.254912e-04,0.459727,0.000445,0.631715,0.378647,0.035161,0.007099,0.277194,0.008129,0.933973,1.262433e-04,0.487845,0.001067,0.021673
2097,0.610415,0.999087,0.781565,1.511369e-04,0.455337,0.000387,0.617487,0.384455,0.031290,0.034323,0.278546,0.017323,0.929100,9.460000e+09,0.990644,0.005779,0.017979
2106,0.589753,0.998878,0.781440,2.890000e+09,0.463349,0.000426,0.623443,0.383173,0.022419,0.036607,0.278094,0.012328,0.928465,1.782309e-04,0.813712,0.053764,0.017870


In [563]:
bank_rf.predict(X_test.iloc[20:25,:])

array([0, 0, 0, 1, 1])

# KNN Classifier

In [564]:
train_scores_KNN, test_scores_KNN=validation_curve(KNeighborsClassifier(), X_bank_trns, y_bank, param_name='n_neighbors', 
                                           param_range=[1,2,3,4,5,6], cv=4)

In [565]:
train_scores_KNN.mean(axis=1)

array([1.        , 0.99066667, 0.941     , 0.92316667, 0.89266667,
       0.877     ])

In [566]:
train_scores_KNN.mean()

0.9374166666666667

In [567]:
test_scores_KNN.mean()

0.8668333333333335

In [568]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel,y_bank,
                                                  random_state=0)

In [569]:
knn_cls_bank_bal=KNeighborsClassifier(n_neighbors=1)

In [570]:
knn_cls_bank_bal.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=1)

In [571]:
print('knn_cls_bank_bal acc on train: {:.2%}'.format(knn_cls_bank_bal.score(X_train,y_train)))
print('knn_cls_bank_bal acc on test: {:.2%}'.format(knn_cls_bank_bal.score(X_test,y_test)),'\n')

knn_cls_bank_bal acc on train: 97.73%
knn_cls_bank_bal acc on test: 87.40% 



In [572]:
knn_cls_bank_bal.score(X_bank_sel,y_bank)

0.9515

In [573]:
w1=[1,1,0.5,0.4,0.3,0.3,0.6,0.7,1,0.4,0.2,0.5,0.7,0.4,0.1,0.9,0.6]
knn_cls_bank_bal.predict([w1])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([0])

In [574]:
knn_cls_bank_bal.predict(X_bank_sel)

array([0, 1, 1, ..., 1, 0, 0])

In [575]:
len(knn_cls_bank_bal.predict(X_bank_sel))

2000

In [576]:
predict_array = knn_cls_bank_bal.predict(X_bank_sel)
count_non_zero = 0
for i in predict_array:
    if i != 1:
        count_non_zero += 1
print(count_non_zero)      

941


In [577]:
sum(knn_cls_bank_bal.predict(X_bank_sel))

1059

In [578]:
X_bank_sel.describe()

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000
mean,0.603785,0.998843,0.781041,1.844045e+09,0.464422,3.732000e+07,0.631498,0.382959,0.039831,3.480000e+06,0.277368,3.585150e+07,0.919430,2.332955e+09,0.748211,7.206000e+07,0.034478
std,0.014686,0.000993,0.003344,3.190004e+09,0.012146,5.890469e+08,0.014577,0.052685,0.036178,1.100197e+08,0.005311,4.840476e+08,0.034846,2.687308e+09,0.204472,7.652163e+08,0.048426
min,0.449761,0.973424,0.735093,1.000256e-04,0.343818,0.000000e+00,0.525126,0.000000,0.009032,0.000000e+00,0.244753,0.000000e+00,0.728885,0.000000e+00,0.049582,1.015558e-04,0.003946
25%,0.597580,0.998857,0.781286,1.549529e-04,0.459606,3.896315e-04,0.630095,0.371918,0.021935,4.761171e-03,0.276920,2.487101e-03,0.909662,7.248945e-04,0.617815,9.988970e-04,0.017979
50%,0.602099,0.998976,0.781541,2.864894e-04,0.462599,4.406694e-04,0.630612,0.376805,0.029839,1.027135e-02,0.277135,5.344259e-03,0.930045,1.270000e+09,0.792352,2.623103e-03,0.023615
75%,0.609738,0.999044,0.781652,3.202500e+09,0.466664,4.735780e-04,0.631087,0.383465,0.043750,2.309766e-02,0.277511,9.879049e-03,0.938763,3.952500e+09,0.919076,6.552762e-03,0.035823
max,0.665151,0.999650,0.786079,9.980000e+09,0.617039,9.330000e+09,0.892562,1.000000,0.404839,3.480000e+09,0.466868,9.910000e+09,0.998858,1.000000e+10,1.000000,9.650000e+09,0.942729
